In [1]:
from google.colab import drive
import os
import sys

drive.mount('/content/drive')
PROJECT_PATH = '/content/drive/MyDrive/ml/ml_Assignment4'
os.chdir(PROJECT_PATH)
sys.path.append('./src')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import json

from models.simple_cnn import SimpleCNN
from models.deeper_cnn import DeepCNN
from models.resnet_variants import ResNetFER
from data_loader import FERDataset, get_data_transforms, create_data_loaders
from training.trainer import FERTrainer
from training.utils import plot_training_curves
from evaluation.metrics import evaluate_model, plot_confusion_matrix


ModuleNotFoundError: No module named 'models'

In [3]:
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PyTorch version: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4


## RUN

In [5]:
def run_experiment(config, train_loader, val_loader):
    print(f"Running Experiment: {config['experiment_name']}")
    print("=" * 60)

    wandb.init(
        project="facial-expression-recognition",
        name=config['experiment_name'],
        config=config,
        tags=config.get('tags', []),
        reinit=True
    )

    if config['model_name'] == 'SimpleCNN':
        model = SimpleCNN(dropout_rate=config['dropout_rate'])
    elif config['model_name'] == 'DeepCNN':
        model = DeepCNN(dropout_rate=config['dropout_rate'])
    elif config['model_name'] == 'ResNetFER':
        model = ResNetFER(dropout_rate=config['dropout_rate'])
    else:
        raise ValueError(f"Unknown model: {config['model_name']}")

    model = model.to(device)

    total_params = sum(p.numel() for p in model.parameters())
    print(f"Model: {config['model_name']} ({total_params:,} parameters)")
    trainer = FERTrainer(model, device, config)
    wandb.watch(model, log_freq=100)
    start_time = time.time()
    best_val_acc = trainer.train(train_loader, val_loader, config['epochs'])
    training_time = time.time() - start_time
    model.load_state_dict(torch.load('best_model.pth'))
    val_accuracy, val_predictions, val_targets = evaluate_model(model, val_loader, device)

    final_train_acc = trainer.train_accuracies[-1]
    final_val_acc = trainer.val_accuracies[-1]
    train_val_gap = final_train_acc - final_val_acc

    wandb.log({
        "final_train_accuracy": final_train_acc,
        "final_val_accuracy": final_val_acc,
        "train_val_gap": train_val_gap,
        "best_val_accuracy": best_val_acc,
        "total_parameters": total_params,
        "training_time_minutes": training_time/60
    })

    model_path = f'./experiments/{config["experiment_name"]}_model.pth'
    os.makedirs('./experiments', exist_ok=True)

    torch.save({
        'model_state_dict': model.state_dict(),
        'config': config,
        'best_val_accuracy': best_val_acc,
        'final_metrics': {
            'train_accuracy': final_train_acc,
            'val_accuracy': final_val_acc,
            'train_val_gap': train_val_gap
        }
    }, model_path)

    print(f"Experiment completed: {best_val_acc:.2f}% best validation accuracy")
    print(f"Training time: {training_time/60:.1f} minutes")

    wandb.finish()

    return {
        'experiment_name': config['experiment_name'],
        'best_val_acc': best_val_acc,
        'final_val_acc': final_val_acc,
        'train_val_gap': train_val_gap,
        'total_params': total_params,
        'training_time': training_time/60,
        'model_path': model_path
    }

In [6]:
base_config = {
    'batch_size': 32,
    'epochs': 20,
    'patience': 6,
    'optimizer': 'adam',
    'scheduler': 'step',
    'step_size': 6,
    'gamma': 0.1,
    'momentum': 0.9,
    'weight_decay': 1e-4
}

all_results = []


Experiment - Baseline without augmentation

In [7]:
config_1a = base_config.copy()
config_1a.update({
    'experiment_name': 'Exp1a_SimpleCNN_NoAugmentation',
    'model_name': 'SimpleCNN',
    'learning_rate': 1e-3,
    'dropout_rate': 0.5,
    'data_augmentation': False,
    'tags': ['series1', 'no-augmentation', 'simplecnn']
})

In [8]:
train_transform, val_transform = get_data_transforms(augment=config_1a['data_augmentation'])
train_loader_no_aug, val_loader = create_data_loaders(
    './data/raw/train.csv',
    batch_size=config_1a['batch_size'],
    train_transform=train_transform,
    val_transform=val_transform
)

result_1a = run_experiment(config_1a, train_loader_no_aug, val_loader)
all_results.append(result_1a)

Running Experiment: Exp1a_SimpleCNN_NoAugmentation


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: egval20 (gval0) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


...

Validation: 100%|██████████| 180/180 [00:04<00:00, 37.90it/s]


Train Loss: 0.6346, Train Acc: 77.41%
Val Loss: 1.3037, Val Acc: 55.60%

Epoch 20/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:04<00:00, 38.01it/s]


Train Loss: 0.6333, Train Acc: 77.59%
Val Loss: 1.3039, Val Acc: 55.67%
New best model saved with validation accuracy: 55.67%
Experiment completed: 55.67% best validation accuracy
Training time: 9.8 minutes


batch_accuracy,▁▁▂▃▄▄▄▅▅▅▅▅▆▅▆▇█▇▇▇██████████▇█████████
batch_loss,███▆▆▅▅▅▅▅▆▆▆▅▄▂▄▃▂▄▁▃▃▄▂▂▂▃▃▄▃▃▂▁▂▃▂▂▂▂
best_val_accuracy,▁
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
final_train_accuracy,▁
final_val_accuracy,▁
learning_rate,███████████▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_parameters,▁
train_accuracy,▁▃▄▄▅▅▇▇▇▇██████████
train_loss,█▆▆▅▄▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train_val_gap,▁


Experiment - Baseline with augmentation

In [9]:
config_1b = base_config.copy()
config_1b.update({
    'experiment_name': 'Exp1b_SimpleCNN_WithAugmentation',
    'model_name': 'SimpleCNN',
    'learning_rate': 1e-3,
    'dropout_rate': 0.5,
    'data_augmentation': True,
    'tags': ['series1', 'augmentation', 'simplecnn']
})

In [10]:
train_transform_aug, val_transform = get_data_transforms(augment=config_1b['data_augmentation'])
train_loader_aug, val_loader = create_data_loaders(
    './data/raw/train.csv',
    batch_size=config_1b['batch_size'],
    train_transform=train_transform_aug,
    val_transform=val_transform
)

result_1b = run_experiment(config_1b, train_loader_aug, val_loader)
all_results.append(result_1b)

Running Experiment: Exp1b_SimpleCNN_WithAugmentation


Model: SimpleCNN (4,741,511 parameters)

Epoch 1/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:04<00:00, 37.87it/s]


Train Loss: 1.6079, Train Acc: 36.39%
Val Loss: 1.4382, Val Acc: 44.73%
New best model saved with validation accuracy: 44.73%

Epoch 2/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:04<00:00, 37.88it/s]


...

Validation: 100%|██████████| 180/180 [00:06<00:00, 27.01it/s]


Train Loss: 0.5553, Train Acc: 80.64%
Val Loss: 1.2704, Val Acc: 56.82%

Epoch 19/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:06<00:00, 27.03it/s]


Train Loss: 0.5562, Train Acc: 80.72%
Val Loss: 1.2703, Val Acc: 56.75%
Early stopping triggered after 6 epochs without improvement
Experiment completed: 57.06% best validation accuracy
Training time: 9.4 minutes


batch_accuracy,▁▂▂▂▂▄▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇█▇▇▇▇▇▇▇▅▇▇
batch_loss,▇▇█▆▆▆▇▆▆▆▆▄▃▃▄▄▃▃▄▅▃▃▃▄▄▂▄▁▁▂▄▂▂▂▂▃▂▃▃▂
best_val_accuracy,▁
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
final_train_accuracy,▁
final_val_accuracy,▁
learning_rate,████████████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
total_parameters,▁
train_accuracy,▁▃▃▄▅▅▇▇▇▇▇████████
train_loss,█▇▆▅▄▄▃▂▂▂▂▁▁▁▁▁▁▁▁
train_val_gap,▁


In [11]:
print(f"RESULTS SUMMARY:")
print(f"No Augmentation: {result_1a['best_val_acc']:.2f}% (gap: {result_1a['train_val_gap']:.2f}%)")
print(f"With Augmentation: {result_1b['best_val_acc']:.2f}% (gap: {result_1b['train_val_gap']:.2f}%)")

RESULTS SUMMARY:
No Augmentation: 55.67% (gap: 21.92%)
With Augmentation: 57.06% (gap: 23.97%)


In [13]:
train_loader, val_loader = train_loader_aug, val_loader

SimpleCNN

In [14]:
config_2a = base_config.copy()
config_2a.update({
    'experiment_name': 'Exp2a_SimpleCNN_Architecture',
    'model_name': 'SimpleCNN',
    'learning_rate': 1e-3,
    'dropout_rate': 0.5,
    'data_augmentation': use_augmentation,
    'tags': ['series2', 'architecture', 'simplecnn']
})

result_2a = run_experiment(config_2a, train_loader, val_loader)
all_results.append(result_2a)

Running Experiment: Exp2a_SimpleCNN_Architecture


Model: SimpleCNN (4,741,511 parameters)

Epoch 1/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:06<00:00, 26.98it/s]


Train Loss: 1.5803, Train Acc: 38.17%
Val Loss: 1.3880, Val Acc: 47.13%
New best model saved with validation accuracy: 47.13%

Epoch 2/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:06<00:00, 26.63it/s]


...

Train Loss: 0.5282, Train Acc: 81.54%
Val Loss: 1.2869, Val Acc: 57.48%

Epoch 17/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:04<00:00, 36.89it/s]

Train Loss: 0.5237, Train Acc: 81.53%
Val Loss: 1.2906, Val Acc: 57.45%
Early stopping triggered after 6 epochs without improvement


Experiment completed: 57.59% best validation accuracy
Training time: 8.3 minutes


batch_accuracy,▁▃▅▅▅▇▆▆▆▆▆▆█▇▇█▇▇▆▇▇▇▇█████████████████
batch_loss,██▇▆▆▅▆▅▃▅▄▃▅▄▄▃▂▄▃▄▃▅▁▃▃▂▂▁▃▁▂▂▃▂▂▂▃▂▁▂
best_val_accuracy,▁
epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
final_train_accuracy,▁
final_val_accuracy,▁
learning_rate,███████████████▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_parameters,▁
train_accuracy,▁▃▃▄▅▅▇▇▇▇███████
train_loss,█▇▆▅▄▄▂▂▂▂▁▁▁▁▁▁▁
train_val_gap,▁


DeepCNN

In [15]:
config_2b = base_config.copy()
config_2b.update({
    'experiment_name': 'Exp2b_DeepCNN_Architecture',
    'model_name': 'DeepCNN',
    'learning_rate': 1e-3,
    'dropout_rate': 0.5,
    'data_augmentation': use_augmentation,
    'tags': ['series2', 'architecture', 'deepcnn']
})

result_2b = run_experiment(config_2b, train_loader, val_loader)
all_results.append(result_2b)

Running Experiment: Exp2b_DeepCNN_Architecture


Model: DeepCNN (1,702,087 parameters)

Epoch 1/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:05<00:00, 32.07it/s]


Train Loss: 4.2086, Train Acc: 23.89%
Val Loss: 1.8053, Val Acc: 25.83%
New best model saved with validation accuracy: 25.83%

Epoch 2/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:08<00:00, 22.03it/s]


...

Train Loss: 1.8220, Train Acc: 24.94%
Val Loss: 1.8027, Val Acc: 25.83%

Epoch 7/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:04<00:00, 37.69it/s]

Train Loss: 1.8159, Train Acc: 24.98%
Val Loss: 1.8022, Val Acc: 25.83%
Early stopping triggered after 6 epochs without improvement


Experiment completed: 25.83% best validation accuracy
Training time: 3.7 minutes


batch_accuracy,▁▂▂▃▃▃▂▃▃▃▃▃▇▃▃▃▃▃▃█▃▃▃▃▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
batch_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_val_accuracy,▁
epoch,▁▂▃▅▆▇█
final_train_accuracy,▁
final_val_accuracy,▁
learning_rate,███████████████████████████████████▁▁▁▁▁
total_parameters,▁
train_accuracy,▁▄▇████
train_loss,█▁▁▁▁▁▁
train_val_gap,▁


ResNetFER

In [16]:
config_2c = base_config.copy()
config_2c.update({
    'experiment_name': 'Exp2c_ResNetFER_Architecture',
    'model_name': 'ResNetFER',
    'learning_rate': 1e-3,
    'dropout_rate': 0.5,
    'data_augmentation': use_augmentation,
    'tags': ['series2', 'architecture', 'resnet']
})

result_2c = run_experiment(config_2c, train_loader, val_loader)
all_results.append(result_2c)

Running Experiment: Exp2c_ResNetFER_Architecture


Model: ResNetFER (2,778,311 parameters)

Epoch 1/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:05<00:00, 31.31it/s]


Train Loss: 3.7200, Train Acc: 27.58%
Val Loss: 1.4983, Val Acc: 43.96%
New best model saved with validation accuracy: 43.96%

Epoch 2/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:04<00:00, 36.36it/s]


...

Train Loss: 0.1586, Train Acc: 94.96%
Val Loss: 2.1279, Val Acc: 58.00%

Epoch 14/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:04<00:00, 36.54it/s]

Train Loss: 0.1135, Train Acc: 96.26%
Val Loss: 2.3765, Val Acc: 57.92%
Early stopping triggered after 6 epochs without improvement


Experiment completed: 59.31% best validation accuracy
Training time: 7.6 minutes


batch_accuracy,▁▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▆▅▅▆▆▅▆▆▆▇▇▇▇▇▇███▇█████
batch_loss,█▅▅▅▄▃▃▃▃▃▂▃▂▃▃▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▂▁▁▁▁▁▁
best_val_accuracy,▁
epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
final_train_accuracy,▁
final_val_accuracy,▁
learning_rate,███████████████████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
total_parameters,▁
train_accuracy,▁▂▃▃▄▄▅▅▆▆▇▇██
train_loss,█▄▄▃▃▃▃▂▂▂▂▁▁▁
train_val_gap,▁


In [17]:
print(f"SERIES 2 RESULTS SUMMARY:")
print(f"SimpleCNN: {result_2a['best_val_acc']:.2f}% ({result_2a['total_params']:,} params)")
print(f"DeepCNN: {result_2b['best_val_acc']:.2f}% ({result_2b['total_params']:,} params)")
print(f"ResNetFER: {result_2c['best_val_acc']:.2f}% ({result_2c['total_params']:,} params)")


SERIES 2 RESULTS SUMMARY:
SimpleCNN: 57.59% (4,741,511 params)
DeepCNN: 25.83% (1,702,087 params)
ResNetFER: 59.31% (2,778,311 params)


In [18]:
best_arch_result = max([result_2a, result_2b, result_2c], key=lambda x: x['best_val_acc'])
best_model_name = best_arch_result['experiment_name'].split('_')[1]

print(f"Using best architecture: {best_model_name}")

learning_rates = [1e-4, 1e-3, 1e-2]

for i, lr in enumerate(learning_rates):
    config_3 = base_config.copy()
    config_3.update({
        'experiment_name': f'Exp3{chr(97+i)}_{best_model_name}_LR_{lr:.0e}',
        'model_name': best_model_name,
        'learning_rate': lr,
        'dropout_rate': 0.5,
        'data_augmentation': use_augmentation,
        'tags': ['series3', 'learning_rate', best_model_name.lower()]
    })

    result = run_experiment(config_3, train_loader, val_loader)
    all_results.append(result)

    print(f"LR {lr:.0e}: {result['best_val_acc']:.2f}%")

Using best architecture: ResNetFER
Running Experiment: Exp3a_ResNetFER_LR_1e-04


Model: ResNetFER (2,778,311 parameters)

Epoch 1/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:06<00:00, 29.88it/s]


Train Loss: 6.7306, Train Acc: 21.23%
Val Loss: 2.3095, Val Acc: 33.25%
New best model saved with validation accuracy: 33.25%

Epoch 2/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:04<00:00, 36.35it/s]


...

Validation: 100%|██████████| 180/180 [00:06<00:00, 27.28it/s]

Train Loss: 0.1316, Train Acc: 96.33%
Val Loss: 1.8613, Val Acc: 58.06%
Early stopping triggered after 6 epochs without improvement


Experiment completed: 58.96% best validation accuracy
Training time: 10.6 minutes


batch_accuracy,▁▁▂▂▂▂▂▃▃▃▃▃▄▃▄▄▄▄▄▅▅▅▆▅▅▅▅▅▆▇▇▇▇▇▇█████
batch_loss,█▂▂▂▂▂▂▁▂▁▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_val_accuracy,▁
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
final_train_accuracy,▁
final_val_accuracy,▁
learning_rate,██████████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_parameters,▁
train_accuracy,▁▂▃▃▃▃▄▄▄▅▅▆▇▇▇▇███
train_loss,█▅▅▄▄▄▄▄▃▃▃▃▂▂▂▁▁▁▁
train_val_gap,▁


LR 1e-02: 58.96%


In [ ]:
best_lr = 1e-2

Adam vs SGD


Adam

In [25]:
config_4a = base_config.copy()
config_4a.update({
    'experiment_name': f'Exp4a_{best_model_name}_Adam',
    'model_name': best_model_name,
    'learning_rate': best_lr,
    'dropout_rate': 0.5,
    'optimizer': 'adam',
    'data_augmentation': use_augmentation,
    'tags': ['series4', 'optimizer', 'adam']
})

result_4a = run_experiment(config_4a, train_loader, val_loader)
all_results.append(result_4a)

Running Experiment: Exp4a_ResNetFER_Adam


Model: ResNetFER (2,778,311 parameters)

Epoch 1/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:06<00:00, 27.98it/s]


Train Loss: 2.8511, Train Acc: 23.50%
Val Loss: 1.7754, Val Acc: 26.34%
New best model saved with validation accuracy: 26.34%

Epoch 2/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:06<00:00, 27.55it/s]


...

Validation: 100%|██████████| 180/180 [00:05<00:00, 30.17it/s]


Train Loss: 0.5601, Train Acc: 80.31%
Val Loss: 1.4304, Val Acc: 56.61%

Epoch 19/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:07<00:00, 25.44it/s]


Train Loss: 0.5045, Train Acc: 82.71%
Val Loss: 1.4390, Val Acc: 56.80%
Early stopping triggered after 6 epochs without improvement
Experiment completed: 56.99% best validation accuracy
Training time: 10.7 minutes


batch_accuracy,▁▁▂▂▂▃▃▃▃▃▄▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█▇▇▇▇
batch_loss,█▄▄▄▄▃▃▄▃▃▄▃▃▄▂▂▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▁▁▁▂▁
best_val_accuracy,▁
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
final_train_accuracy,▁
final_val_accuracy,▁
learning_rate,██████████████▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_parameters,▁
train_accuracy,▁▁▂▃▃▃▄▄▅▅▅▆▇▇▇▇███
train_loss,█▅▅▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁
train_val_gap,▁


SGD

In [27]:
config_4b = base_config.copy()
config_4b.update({
    'experiment_name': f'Exp4b_{best_model_name}_SGD',
    'model_name': best_model_name,
    'learning_rate': 1e-2 * 10,
    'dropout_rate': 0.5,
    'optimizer': 'sgd',
    'momentum': 0.9,
    'data_augmentation': use_augmentation,
    'tags': ['series4', 'optimizer', 'sgd']
})

result_4b = run_experiment(config_4b, train_loader, val_loader)
all_results.append(result_4b)

Running Experiment: Exp4b_ResNetFER_SGD


Model: ResNetFER (2,778,311 parameters)

Epoch 1/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:05<00:00, 35.99it/s]


Train Loss: 2.3348, Train Acc: 23.70%
Val Loss: 1.7935, Val Acc: 25.87%
New best model saved with validation accuracy: 25.87%

Epoch 2/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:05<00:00, 35.38it/s]


...

Validation: 100%|██████████| 180/180 [00:07<00:00, 25.36it/s]


Train Loss: 1.2720, Train Acc: 51.98%
Val Loss: 1.3639, Val Acc: 47.52%

Epoch 20/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:06<00:00, 29.43it/s]


Train Loss: 1.2686, Train Acc: 52.21%
Val Loss: 1.3642, Val Acc: 48.16%
New best model saved with validation accuracy: 48.16%
Experiment completed: 48.16% best validation accuracy
Training time: 10.9 minutes


batch_accuracy,▁▁▁▂▂▂▂▂▂▂▁▃▃▄▄▅▅▅▅▆▆▆▆▆█▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇
batch_loss,█▇█▇█▅▅▅▆▅▄▅▂▂▂▅▄▆▅▅▃▃▃▃▁▃▂▂▄▁▄▃▂▄▃▁▃▃▃▆
best_val_accuracy,▁
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
final_train_accuracy,▁
final_val_accuracy,▁
learning_rate,████████████▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_parameters,▁
train_accuracy,▁▁▁▂▃▄▅▆▆▆▆▇▇▇██████
train_loss,█▅▄▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
train_val_gap,▁


In [29]:
best_optimizer = 'adam'
best_optimizer_lr = 1e-2

print(f" Using best optimizer: {best_optimizer} with LR: {best_optimizer_lr}")

dropout_rates = [0.3, 0.5, 0.7]

for i, dropout in enumerate(dropout_rates):
    config_5 = base_config.copy()
    config_5.update({
        'experiment_name': f'Exp5{chr(97+i)}_{best_model_name}_Dropout_{dropout:.1f}',
        'model_name': best_model_name,
        'learning_rate': best_optimizer_lr,
        'dropout_rate': dropout,
        'optimizer': best_optimizer,
        'data_augmentation': use_augmentation,
        'tags': ['series5', 'regularization', 'dropout']
    })

    result = run_experiment(config_5, train_loader, val_loader)
    all_results.append(result)

    print(f"Dropout {dropout}: {result['best_val_acc']:.2f}%")


 Using best optimizer: adam with LR: 0.01
Running Experiment: Exp5a_ResNetFER_Dropout_0.3


Model: ResNetFER (2,778,311 parameters)

Epoch 1/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:05<00:00, 35.55it/s]


Train Loss: 2.5903, Train Acc: 23.48%
Val Loss: 1.7621, Val Acc: 28.76%
New best model saved with validation accuracy: 28.76%

Epoch 2/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:06<00:00, 27.85it/s]


...

Validation: 100%|██████████| 180/180 [00:05<00:00, 31.19it/s]

Train Loss: 0.6271, Train Acc: 78.42%
Val Loss: 1.3268, Val Acc: 56.68%
Early stopping triggered after 6 epochs without improvement


Experiment completed: 57.45% best validation accuracy
Training time: 10.6 minutes


batch_accuracy,▁▁▁▁▁▁▁▂▂▃▃▃▂▄▄▆▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆█▇▇█▇
batch_loss,█▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_val_accuracy,▁
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
final_train_accuracy,▁
final_val_accuracy,▁
learning_rate,███████████▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_parameters,▁
train_accuracy,▁▁▁▁▂▃▄▄▄▅▅▅▆▇▇▇▇██
train_loss,█▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁
train_val_gap,▁


Dropout 0.7: 57.45%


## Final Model

In [30]:
best_model_name = 'ResNetFER'
best_lr = 1e-2
best_dropout = 0.3
best_optimizer = 'adam'

In [31]:
final_config = base_config.copy()
final_config.update({
    'experiment_name': 'Final_Best_Model_Extended',
    'model_name': best_model_name,
    'learning_rate': best_lr,
    'dropout_rate': best_dropout,
    'optimizer': best_optimizer,
    'epochs': 20,
    'patience': 12,
    'data_augmentation': use_augmentation,
    'tags': ['final', 'best_model', 'extended_training']
})
final_result = run_experiment(final_config, train_loader, val_loader)
all_results.append(final_result)

Running Experiment: Final_Best_Model_Extended


Model: ResNetFER (2,778,311 parameters)

Epoch 1/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:05<00:00, 32.99it/s]


Train Loss: 2.5078, Train Acc: 24.17%
Val Loss: 1.7133, Val Acc: 30.71%
New best model saved with validation accuracy: 30.71%

Epoch 2/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:05<00:00, 35.00it/s]


...

Validation: 100%|██████████| 180/180 [00:05<00:00, 35.98it/s]


Train Loss: 0.1749, Train Acc: 94.75%
Val Loss: 1.8830, Val Acc: 57.46%

Epoch 20/20
--------------------------------------------------


Validation: 100%|██████████| 180/180 [00:05<00:00, 35.60it/s]


Train Loss: 0.1673, Train Acc: 94.94%
Val Loss: 1.8689, Val Acc: 57.59%
Experiment completed: 58.02% best validation accuracy
Training time: 11.0 minutes


batch_accuracy,▁▂▂▂▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█▇█████████
batch_loss,█▆▅▅▆▅▅▅▅▅▄▄▅▅▃▃▃▃▂▃▃▄▃▃▂▁▁▁▁▂▂▁▁▁▁▂▁▁▁▁
best_val_accuracy,▁
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
final_train_accuracy,▁
final_val_accuracy,▁
learning_rate,█████████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_parameters,▁
train_accuracy,▁▂▃▃▃▃▄▄▅▅▅▆▇▇▇▇████
train_loss,█▅▅▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁
train_val_gap,▁


In [32]:
wandb.init(
    project="facial-expression-recognition",
    name="comprehensive_results_analysis",
    job_type="analysis"
)

In [33]:
final_model_path = final_result['model_path']
final_model_data = torch.load(final_model_path)

enhanced_model_path = './experiments/FINAL_BEST_MODEL.pth'

overall_best = max(all_results, key=lambda x: x['best_val_acc'])

enhanced_model_data = {
    'model_state_dict': final_model_data['model_state_dict'],
    'final_config': final_config,
    'best_val_accuracy': final_result['best_val_acc'],
    'all_experiment_results': all_results,
    'experiment_summary': {
        'total_experiments': len(all_results),
        'best_overall_accuracy': overall_best['best_val_acc'],
        'best_experiment_name': overall_best['experiment_name'],
        'final_model_accuracy': final_result['best_val_acc'],
        'model_architecture': final_config['model_name'],
        'final_hyperparameters': {
            'learning_rate': final_config['learning_rate'],
            'dropout_rate': final_config['dropout_rate'],
            'optimizer': final_config['optimizer'],
            'data_augmentation': final_config['data_augmentation']
        }
    },
    'training_history': {
        'train_accuracies': final_model_data.get('train_accuracies', []),
        'val_accuracies': final_model_data.get('val_accuracies', []),
        'train_losses': final_model_data.get('train_losses', []),
        'val_losses': final_model_data.get('val_losses', [])
    },
    'for_next_notebook': {
        'model_name': final_config['model_name'],
        'dropout_rate': final_config['dropout_rate'],
        'data_augmentation': final_config['data_augmentation'],
        'best_hyperparameters': final_config
    }
}

torch.save(enhanced_model_data, enhanced_model_path)

print(f"Enhanced model saved to: {enhanced_model_path}")
print(f"Final model accuracy: {final_result['best_val_acc']:.2f}%")
print(f"Architecture: {final_config['model_name']}")

wandb.init(
    project="facial-expression-recognition",
    name="final_best_model_artifact",
    job_type="model_registry",
    tags=["final_model", "best_model", "artifact"]
)

final_artifact = wandb.Artifact(
    name="final_best_fer_model",
    type="model",
    description=f"""
    Final best FER model from systematic experimentation:
    - Architecture: {final_config['model_name']}
    - Validation Accuracy: {final_result['best_val_acc']:.2f}%
    - Total Experiments: {len(all_results)}
    - Best Overall: {overall_best['experiment_name']} ({overall_best['best_val_acc']:.2f}%)
    - Hyperparameters: LR={final_config['learning_rate']}, Dropout={final_config['dropout_rate']}
    - Data Augmentation: {final_config['data_augmentation']}
    """,
    metadata={
        "accuracy": final_result['best_val_acc'],
        "architecture": final_config['model_name'],
        "total_experiments": len(all_results),
        "best_hyperparameters": final_config,
        "experiment_series_completed": ["data_augmentation", "architecture", "learning_rate", "optimizer", "regularization"]
    }
)

final_artifact.add_file(enhanced_model_path, name="final_best_model.pth")
final_artifact.add_file('./experiments/all_experiment_results.json', name="experiment_results.json")

wandb.log_artifact(final_artifact)
wandb.log({
    "final_model_accuracy": final_result['best_val_acc'],
    "total_experiments_completed": len(all_results),
    "best_architecture": final_config['model_name'],
    "experiment_success": True,
    "ready_for_final_analysis": True
})


Enhanced model saved to: ./experiments/FINAL_BEST_MODEL.pth
Final model accuracy: 58.02%
Architecture: ResNetFER


ValueError: Path is not a file: './experiments/all_experiment_results.json'

In [ ]:
summary_table = wandb.Table(
    columns=["Experiment", "Model", "Val_Accuracy", "Train_Val_Gap", "Parameters", "Training_Time"],
    data=[[r['experiment_name'], r['experiment_name'].split('_')[1] if '_' in r['experiment_name'] else 'Unknown',
           r['best_val_acc'], r['train_val_gap'], r['total_params'], r['training_time']]
          for r in all_results]
)

wandb.log({
    "experiment_summary": summary_table,
    "results_visualization": wandb.Image(plt),
    "best_experiment": best_result['experiment_name'],
    "best_accuracy": best_result['best_val_acc'],
    "total_experiments_run": len(all_results)
})


In [4]:
wandb.finish()